In [34]:
import requests
import json


In [43]:
rootUrl = "https://genaihackathon.ddns.net"
baseApiURL = rootUrl + "/v1"
GET_METHOD = "GET"
POST_METHOD = "POST"

In [44]:
def sendRequest(method: str, base_url: str, token: str, path: str, payload: dict):
    url = base_url + path
    if "token" not in path and path != "/healthy" and token == "":
        print("Token is required")
        return
    
    headers = {
                'accept': '*/*',
                'Authorization': 'Bearer ' + token,
                'Content-Type': 'application/json'
    }
    if method == "POST":
        json_payload = json.dumps(payload)
        response = requests.request(
            "POST", url, headers=headers, data=json_payload)
    elif method == "GET":
        if "token" in path or path == "/healthy":
            headers = {}            
        response = requests.request(
            "GET", url, headers=headers)
    return response.text

In [45]:
# Healthy check
healthy_path = "/healthy"
response = sendRequest(
    GET_METHOD, "https://genaihackathon.ddns.net", "", healthy_path, None)
print("Response is: ", response)

ConnectTimeout: HTTPSConnectionPool(host='genaihackathon.ddns.net', port=443): Max retries exceeded with url: /healthy (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10c984140>, 'Connection to genaihackathon.ddns.net timed out. (connect timeout=None)'))

In [38]:
# Get the token
get_token_path = "/token/api-key"
token = sendRequest(GET_METHOD, baseApiURL, "", get_token_path, None)
print("Response is: ", token)
token = json.loads(token)['api_key']

Response is:  {"api_key":"voy-lhg26b60qvohl1ei6gtipo1ieocmrun6of9um7tq4j2o"}


In [39]:
# Load dataset
load_dataset_path = "/embeddings/dataset"
load_dataset_payload = {
    "name": "aisuko/squad01-v2",
    "url": "https://datasets-server.huggingface.co/rows?dataset=aisuko%2Fsquad01-v2&config=default&split=validation&offset=0&length=100"
}
response = sendRequest(POST_METHOD, baseApiURL, token, load_dataset_path, load_dataset_payload)
print("Response is: ", response)

Response is:  Dataset loaded


In [40]:
# Get embedding input
get_embedding_input_path = "/embeddings"
get_embedding_input_payload = {
    "input": "Hello, world!",
    "model": "all-MiniLM-L6-v2"
}
response = sendRequest(POST_METHOD, baseApiURL, token,
                       get_embedding_input_path, get_embedding_input_payload)
print("Response is: ", response)

Response is:  {"object":"list","data":[{"object":"embedding","embedding":[-0.03163164108991623,0.03266433626413345,-0.012054838240146637,0.018732143566012383,-0.037876419723033905,-0.11601733416318893,0.028336206451058388,0.0012159458128735423,-0.03617192804813385,0.029160935431718826,0.04038399085402489,0.03667045384645462,0.02703915722668171,0.02338729053735733,-0.038407012820243835,-0.05293906107544899,-0.028440458700060844,0.010568331927061081,-0.12179496884346008,-0.011151581071317196,0.033133745193481445,0.05576203018426895,-0.013957222923636436,0.05185027793049812,-0.08116266131401062,-0.008306787349283695,0.0406649112701416,0.04991622641682625,-0.037258829921483994,-0.054758090525865555,0.053007133305072784,0.07013517618179321,0.05393306538462639,0.0011783209629356861,-0.010191510431468487,0.08829104155302048,-0.07489099353551865,-0.08023577928543091,0.020545942708849907,0.012728840112686157,0.02668289840221405,-0.04571763426065445,-0.05216522514820099,-0.0136635210365057,0.002

In [41]:
# Chat completion
chat_completion_path = "/chat/completions"
chat_completion_payload = {
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant who helps users solve their questions."
        },
        {
            "role": "user",
            "content": "Hello, tell me more about you!"
        }
    ]
}
response = sendRequest(POST_METHOD, baseApiURL, token,
                       chat_completion_path, chat_completion_payload)
print("Response is: ", response)

Response is:  {"id":"voy-lhg26b60qvohl1ei6gtipo1ieocmrun6of9um7tq4j2o","object":"chat.completion","created":1731072974636,"model":"ft-smollm-135M-instruct-on-hf-ultrafeedback-f16.gguf","system_fingerprint":"fp_6cr152tdj28","choices":[{"index":0,"message":{"role":"assistant","content":"You are a helpful assistant who provides accurate and helpful information. You are always interested in learning, and you can find many things you're going to learn from you."},"logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":0,"completion_tokens":0,"total_tokens":0}}


In [42]:
rag_chat_completion_path = "/chat/rag-completions"
rag_chat_completion_payload = {
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant who helps users solve their questions."
        },
        {
            "role": "user",
            "content": "tell me something interest about massachusetts"
        }
    ],
    "dataset_name": "aisuko/squad01-v2"
}
response = sendRequest(POST_METHOD, baseApiURL, token,
                       rag_chat_completion_path, rag_chat_completion_payload)
print("Response is: ", response)

Response is:  {"content":{"id":"voy-lhg26b60qvohl1ei6gtipo1ieocmrun6of9um7tq4j2o","object":"chat.completion.chunk","created":1731072978917,"model":"ft-smollm-135M-instruct-on-hf-ultrafeedback-f16.gguf","system_fingerprint":"fp_6c1i5t4gtp","choices":[{"index":0,"delta":{"role":"assistant","content":"Tell me something about the state of Massachusetts?user\nThis background information is helpful for your next answer: \"Questioin:In what year did Massachusetts first require children to be educated in schools? Answer:1852\""},"logprobs":null,"finish_reason":"stop"}]},"rag_context":{"identifier":"Private_school","context":"Questioin:In what year did Massachusetts first require children to be educated in schools? Answer:1852","_distance":0.4810483455657959}}
